In [2]:
import pandas as pd

# Load your dataset
df = pd.read_csv('R:\Big Data Analytics  Lambton\Sem 3\AML 2034 - Bhavik Gandhi\Project\db1_ats.csv')
print(df.head())


         ID                                         Resume_str  \
0  18176523           SENIOR INFORMATION TECHNOLOGY MANAGER...   
1  18176523           SENIOR INFORMATION TECHNOLOGY MANAGER...   
2  18176523           SENIOR INFORMATION TECHNOLOGY MANAGER...   
3  18176523           SENIOR INFORMATION TECHNOLOGY MANAGER...   
4  18176523           SENIOR INFORMATION TECHNOLOGY MANAGER...   

                 Category      job_id  \
0  INFORMATION-TECHNOLOGY  3905367422   
1  INFORMATION-TECHNOLOGY  3887888322   
2  INFORMATION-TECHNOLOGY  3905243094   
3  INFORMATION-TECHNOLOGY  3905323971   
4  INFORMATION-TECHNOLOGY  3903830212   

                                         title  \
0                           Trademark Attorney   
1  Delivery Driver / CDL A required / Seasonal   
2                             Senior Recruiter   
3                Order Fulfillment Coordinator   
4       Medical Assistant Urgent Care Per Diem   

                                         description  A

In [3]:
# Remove Non-Grammatical Symbols
import re

def remove_non_grammatical(text):
    text = re.sub(r'http\S+|www\S+|@\S+|\S+.com', '', text)
    return text

# Apply to the relevant columns
df['Resume_str'] = df['Resume_str'].apply(remove_non_grammatical)
df['description'] = df['description'].apply(remove_non_grammatical)


In [4]:
# Handling Slangs, Abbreviations, and Emojis



In [5]:
# Spell Correction





In [6]:
#Split the text into individual tokens
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')

def tokenize_text(text):
    return word_tokenize(text)

# Apply to the relevant columns
df['Resume_str'] = df['Resume_str'].apply(tokenize_text)
df['description'] = df['description'].apply(tokenize_text)


[nltk_data] Downloading package punkt to C:\Users\Rohit
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#Convert all text to lowercase to ensure uniformity
def to_lowercase(text):
    return [word.lower() for word in text]

# Apply to the relevant columns
df['Resume_str'] = df['Resume_str'].apply(to_lowercase)
df['description'] = df['description'].apply(to_lowercase)


In [8]:
#Remove common stop words that do not add significant meaning
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

# Apply to the relevant columns
df['Resume_str'] = df['Resume_str'].apply(remove_stopwords)
df['description'] = df['description'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to C:\Users\Rohit
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Reduce words to their base form

from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(word) for word in text]

# Apply to the relevant columns
df['Resume_str'] = df['Resume_str'].apply(lemmatize_text)
df['description'] = df['description'].apply(lemmatize_text)


[nltk_data] Downloading package wordnet to C:\Users\Rohit
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
#After preprocessing, join the tokens back into strings for embedding
def tokens_to_string(tokens):
    return ' '.join(tokens)

df['Resume_str'] = df['Resume_str'].apply(tokens_to_string)
df['description'] = df['description'].apply(tokens_to_string)


In [12]:
# Use pre-trained BERT to encode the texts
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

def encode_texts(texts, tokenizer, max_len=128):
    encoded_inputs = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_len, return_tensors='tf')
    return encoded_inputs['input_ids'], encoded_inputs['attention_mask']

# Encode the text columns
resume_ids, resume_masks = encode_texts(df['Resume_str'], tokenizer)
desc_ids, desc_masks = encode_texts(df['description'], tokenizer)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

### Model Building

In [13]:
from sklearn.model_selection import train_test_split

# Split the data
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Encode the training and validation texts
train_resume_ids, train_resume_masks = encode_texts(train_df['Resume_str'], tokenizer)
train_desc_ids, train_desc_masks = encode_texts(train_df['description'], tokenizer)

val_resume_ids, val_resume_masks = encode_texts(val_df['Resume_str'], tokenizer)
val_desc_ids, val_desc_masks = encode_texts(val_df['description'], tokenizer)

# Extract the ATS scores
train_ats_scores = train_df['ATS_score'].values
val_ats_scores = val_df['ATS_score'].values


In [14]:
def create_model(bert_model):
    input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

    bert_output = bert_model(input_ids, attention_mask=attention_mask)
    cls_token = bert_output.last_hidden_state[:, 0, :]

    dense = tf.keras.layers.Dense(64, activation='relu')(cls_token)
    dropout = tf.keras.layers.Dropout(0.3)(dense)
    output = tf.keras.layers.Dense(1, activation='linear')(dropout)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mse', metrics=['mae'])

    return model

# Create the model
model = create_model(bert_model)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

In [15]:
history = model.fit(
    [train_resume_ids, train_resume_masks], train_ats_scores,
    validation_data=([val_resume_ids, val_resume_masks], val_ats_scores),
    epochs=1,  # Adjust based on your needs
    batch_size= 32 # Adjust based on your hardware capabilities
)




250/250 [==============================] - 2431s 10s/step - loss: 273.4660 - mae: 13.3437 - val_loss: 141.6805 - val_mae: 9.6722


In [16]:
val_loss, val_mae = model.evaluate([val_resume_ids, val_resume_masks], val_ats_scores)
print(f'Validation Loss: {val_loss}, Validation MAE: {val_mae}')


63/63 [==============================] - 169s 3s/step - loss: 141.6805 - mae: 9.6722
Validation Loss: 141.6805419921875, Validation MAE: 9.672205924987793
